In [1]:
import torch
from FusionNet import FusionGenerator
from model_utils_sw import MaskToBinary, DiceCoefficient

from torchvision import transforms
import torchvision.utils as v_utils
from torch.autograd import Variable
import scipy.misc

import os
import numpy as np
from PIL import Image

#=====================
# hyperparameter
INPUT_SIZE=256
#=====================

#=====================
# functions to use
# 1. image2tensor

#=====================
def image2tensor(img, cuda=False):
    img_array=np.asarray(img)
    img_array=np.expand_dims(np.expand_dims(img_array, 0), 0)
    if cuda:
        img_array=torch.Tensor(img_array).cuda()
    else:
        img_array=torch.Tensor(img_array)
    return img_array

#=====================

# load model
# model=FusionGenerator(input_nc=1, output_nc=1, ngf=32)
# model.load_state_dict(torch.load('./FusionNet.pkl'))
model=torch.load('./FusionNet.pkl').cuda()

# get data path and load data
original_dir='../segmentation_testset/TCGA_original/'
mask_dir='../segmentation_testset/TCGA_mask/'
save_dir='../segmentation_testset/TCGA_gen_256/'
save_label_dir='../segmentation_testset/TCGA_mask_256/'
save_resize_origin_dir='../segmentation_testset/TCGA_original_256/'

subjects=os.listdir(original_dir)

/home/ccids-sw/anaconda3/envs/testenv/lib/python3.5/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [2]:
# pass through pre-trained model and save image to directory

for subject in subjects:
    dice_sum=0; k=0
    images=os.listdir(os.path.join(original_dir, subject))
    for i, image in enumerate(images):
        
        # get one image
        one_img=os.path.join(original_dir, subject, image)
        
        #resizing image
        one_img=Image.open(one_img)
        one_img=one_img.resize((INPUT_SIZE, INPUT_SIZE))
        
        #change to pytorch tensor
        one_img=image2tensor(one_img, cuda=True)
        
        #pass through model
        result_img=model(one_img)
        
        # if there isn't subjecy dir in save dir, make directory for subject
        if not os.path.exists(os.path.join(save_dir, subject)):
            os.mkdir(os.path.join(save_dir, subject))
            
        if not os.path.exists(os.path.join(save_label_dir, subject)):
            os.mkdir(os.path.join(save_label_dir, subject))
            
        if not os.path.exists(os.path.join(save_resize_origin_dir, subject)):
            os.mkdir(os.path.join(save_resize_origin_dir, subject))
            
        # change to numpy array again and save to suitable folder
        result_array=MaskToBinary(np.asarray(result_img[0].cpu().data))
        image_save_path=os.path.join(save_dir, subject, image)
        scipy.misc.imsave(image_save_path, result_array[0])
        
        # open mask file and calculate dice score
        # resize label image and save
        label_img=Image.open(os.path.join(mask_dir, subject, image)).resize((INPUT_SIZE, INPUT_SIZE))
        label_array=np.asarray(label_img)
        mask_save_path=os.path.join(save_label_dir, subject, image)
        scipy.misc.imsave(mask_save_path, label_array)
        
        # resize original image and save
        origin_img=Image.open(os.path.join(original_dir, subject, image)).resize((INPUT_SIZE, INPUT_SIZE))
        origin_array=np.asarray(origin_img)
        origin_save_path=os.path.join(save_resize_origin_dir, subject, image)
        scipy.misc.imsave(origin_save_path, origin_array)
        
        if result_array[0].shape != label_array.shape:
            print('Size mismatch label image size -- {}, result image size -- {} try again!'.format(label_array.shape,result_array[0].shape))

        dice_score=DiceCoefficient(np.expand_dims(label_array,0),  result_array)
#         print(image, dice_score)
        if dice_score != 0.0:
            dice_sum+=dice_score
            k += 1
    print('dice score for {} : {}'.format(subject, dice_sum / k ))
#     break

/home/ccids-sw/anaconda3/envs/testenv/lib/python3.5/site-packages/ipykernel_launcher.py:34: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/home/ccids-sw/anaconda3/envs/testenv/lib/python3.5/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/home/ccids-sw/anaconda3/envs/testenv/lib/python3.5/site-packages/ipykernel_launcher.py:47: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


dice score for DU_5852 : 0.008383833227674579
dice score for HT_8015 : 0.00020138929086568688
dice score for FG_6692 : 0.010061213165839559
dice score for FG_7643 : 0.019234324655200856
dice score for HT_8107 : 0.008859759155418579
dice score for CS_5397 : 0.010083784868964147
dice score for CS_6186 : 0.009550179644536607
dice score for HT_8110 : 0.011634156406827209
dice score for HT_7680 : 0.0002339353523900326
dice score for DU_A5TT : 0.014581874061600828
dice score for CS_6188 : 0.007963955630503639
dice score for HT_7469 : 0.01412323272992354
dice score for HT_A5RC : 0.008282345499551615
dice score for CS_5395 : 0.0086460712257049
dice score for DU_8165 : 0.0018722836573400812
dice score for DU_7014 : 0.009826371320948078
dice score for HT_7882 : 0.005313783263967665
dice score for HT_A61A : 0.012944000723818367
dice score for HT_7860 : 0.013740545516259783
dice score for FG_5963 : 0.02071447117636853
dice score for CS_6669 : 0.008264677369644029
dice score for DU_8162 : 0.0053712